In [2]:
import numpy as np


In [1]:
import copy
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
# sys.path.append("../../../")
from model.mujoco_parser import MuJoCoParserClass
from model.util import sample_xyzs,rpy2r,r2quat,get_interp_const_vel_traj

/home/rilab/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-23 20:13:48,235	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
xml_path = './asset/visualize_realworld_wo_shelf_pringles.xml'
env = MuJoCoParserClass(name='Place task scene: Office table',rel_xml_path=xml_path,VERBOSE=False, MODE='window')
print(env.MODE)

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('avoiding_object_table').pos = np.array([0.38+0.45,0,0])
env.model.body('ur_base').pos = np.array([-0.18,0,0.79])
env.model.body('right_object_table').pos = np.array([-0.05,-0.80,0])
env.model.body('left_object_table').pos = np.array([-0.05,0.80,0])

# Place objects
tray_pos = np.array([ 0.9, 0.0, 0.8])
bowl_red_pos = np.array([ 0.9, 0.3, 0.8])
bowl_blue_pos = np.array([ 0.9, -0.3, 0.8])
env.model.joint(env.model.body('tray_gray').jntadr[0]).qpos0[:3] = tray_pos
env.model.joint(env.model.body('kitchen-bowl-red').jntadr[0]).qpos0[:3] = bowl_red_pos
env.model.joint(env.model.body('kitchen-bowl-blue').jntadr[0]).qpos0[:3] = bowl_blue_pos

env.model.joint(env.model.body('ycb-apple-2').jntadr[0]).qpos0[:3] = bowl_red_pos + np.array([0.0,0.0,0.02])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([0.0,0.03,0.2])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([0, 0, 90])))
env.model.joint(env.model.body('ycb-lemon-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([-0.03,-0.07,0.02])
env.model.joint(env.model.body('ycb-orange-2').jntadr[0]).qpos0[:3] = bowl_blue_pos + np.array([0.0,0.0,0.02])

# Target objects
env.model.joint(env.model.body('ycb-banana').jntadr[0]).qpos0[:3] = np.array([-5.0,-1.5,0.2])
env.model.joint(env.model.body('ycb-apple').jntadr[0]).qpos0[:3] = np.array([0.15, -0.8, 0.8])
env.model.joint(env.model.body('ycb-lemon').jntadr[0]).qpos0[:3] = np.array([0.075, -0.8, 0.8])
env.model.joint(env.model.body('ycb-orange').jntadr[0]).qpos0[:3] = np.array([0.0, -0.8, 0.8])

joint_names = env.rev_joint_names[:6]
idxs_forward = [env.model.joint(joint_name).qposadr[0] for joint_name in env.joint_names[:6]]
idxs_jacobian = [env.model.joint(joint_name).dofadr[0] for joint_name in env.joint_names[:6]]
list1, list2 = env.ctrl_joint_idxs, idxs_forward
idxs_step = []
for i in range(len(list2)):
    if list2[i] in list1:
        idxs_step.append(list1.index(list2[i]))

window


In [3]:
# Origin
capture_q = np.array([np.deg2rad(-90), np.deg2rad(-132.46), np.deg2rad(122.85), np.deg2rad(99.65), np.deg2rad(45), np.deg2rad(-90.02)])


In [18]:
#### 
def rand_in_range(q):
    cos_q = np.cos(q)
    sin_q = np.sin(q)
    refined_q = np.arctan2(sin_q, cos_q)
    return refined_q

In [19]:
def solve_ik(env:MuJoCoParserClass,body_name,p_trgt,R_trgt,IK_P,IK_R,q_init,idxs_forward, idxs_jacobian,
                RESET=False,DO_RENDER=False,render_every=1,th=1*np.pi/180.0,err_th=1e-6,w_weight=1.0, stepsize=1.0, eps=0.1,
                repulse = 30, inc_prefix = None, exc_prefix = None):
    """
        Solve IK
    """
    if RESET:
        env.reset()
    q_backup = env.get_q(joint_idxs=idxs_forward)
    q = q_init.copy()
    env.forward(q=q,joint_idxs=env.idxs_forward)
    q = q[idxs_forward]
    tick = 0
    while True:
        tick = tick + 1
        J,err = env.get_ik_ingredients(
            body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,IK_P=IK_P,IK_R=IK_R, w_weight=w_weight)
        dq = env.damped_ls(J,err,stepsize=stepsize,eps=eps,th=th)
        q = q + dq[idxs_jacobian]
        env.forward(q=q,joint_idxs=idxs_forward)

        p_contacts,f_contacts,geom1s,geom2s,body1s,body2s = env.get_contact_info(must_include_prefix=inc_prefix, must_exclude_prefix=exc_prefix)
        
        body1s_ = [obj_ for obj_ in body1s if obj_ not in ["ur_rg2_gripper_finger1_finger_tip_link","ur_rg2_gripper_finger2_finger_tip_link"]]
        body2s_ = [obj_ for obj_ in body2s if obj_ not in ["ur_rg2_gripper_finger1_finger_tip_link","ur_rg2_gripper_finger2_finger_tip_link"]]
        
        if len(body1s_) > 0:
            q = q - dq[idxs_jacobian] * repulse
        
        # Terminate condition
        err_norm = np.linalg.norm(err)
        if err_norm < err_th:
            break
        # Render
        if DO_RENDER:
            if ((tick-1)%render_every) == 0:
                p_tcp,R_tcp = env.get_pR_body(body_name=body_name)
                env.plot_T(p=p_tcp,R=R_tcp,PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)
                env.plot_T(p=p_trgt,R=R_trgt,PLOT_AXIS=True,axis_len=0.2,axis_width=0.005)
                env.render(render_every=render_every)
    # Back to back-uped position
    q_ik = env.get_q(joint_idxs=idxs_forward)
    env.forward(q=q_backup,joint_idxs=idxs_forward)
    q_ik = rand_in_range(q_ik)
    return q_ik

In [16]:
env.get_p_body(body_name='ur_tcp_link')

array([1.44918831e-01, 5.41244959e-04, 1.14113508e+00])

In [20]:
env.init_viewer(viewer_title='IK solver',viewer_width=1200,viewer_height=800,viewer_hide_menus=True, MODE='window')
env.update_viewer(azimuth=00,distance=2.5,elevation=-30,lookat=[0,0,1.5])
env.update_viewer(VIS_JOINT=False,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
env.reset() # reset

q_init = capture_q.copy()
# q_init = q_ik_target #np.array([0.65, 0.27 , 0.9])
print(q_init)
env.forward(q=q_init,joint_idxs=idxs_forward)
p_target = np.array([0.65, 0.0 , 0.85])
R_target = rpy2r(np.array([-180,0,90])*np.pi/180.0)
# R_target = rpy2r(np.array([-180,0,90])*np.pi/180.0)

q_ik_target = solve_ik(env=env,
    body_name='ur_tcp_link',p_trgt=p_target,R_trgt=R_target,
    IK_P=True,IK_R=True, q_init=capture_q,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    inc_prefix = 'ur', exc_prefix=None,
    RESET=False, DO_RENDER=True, th=1, err_th=1e-3, stepsize=1, w_weight=0.3, render_every=1.0, repulse=10)

print(f"Initial Joint values: {q_init}")
print(f"Solved IK: {q_ik_target}")

# Close viewer
env.close_viewer()
print ("Done.")

[-1.57079633 -2.31186313  2.14413699  1.7392206   0.78539816 -1.57114539]
Initial Joint values: [-1.57079633 -2.31186313  2.14413699  1.7392206   0.78539816 -1.57114539]
Solved IK: [-2.89943624e+00 -2.34831129e+00 -1.68530159e+00 -2.25158631e+00
 -1.32768713e+00  7.17750593e-04]
Done.


In [8]:
# env.close_viewer()

In [60]:
env.idxs_jacobian

[0, 1, 2, 3, 4, 5]

In [65]:
env.step()
q_init = q_ik_target
q_init

array([ 1.99770948e-01, -5.78930392e-01,  1.07079956e+00, -4.90775170e-01,
        1.77120524e+00,  2.43002636e-04])

In [66]:
p_target = np.array([0.9, 0.27 , 0.9])
R_target = rpy2r(np.array([-180,0,90])*np.pi/180.0)

q_ik_target = solve_ik(env=env,
    body_name='ur_tcp_link',p_trgt=p_target,R_trgt=R_target,
    IK_P=True,IK_R=True, q_init=q_init,idxs_forward=[0,1,2,3,4], idxs_jacobian=[0,1,2,3,4],
    inc_prefix = 'ur', exc_prefix=None,
    RESET=False, DO_RENDER=True, th=1, err_th=1e-3, stepsize=1, w_weight=0.3, render_every=1.0, repulse=1)

print(f"Initial Joint values: {q_init}")
print(f"Solved IK: {q_ik_target}")

# Close viewer
env.close_viewer()
print ("Done.")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_44387/4248793273.py", line 4, in <module>
    q_ik_target = solve_ik(env=env,
  File "/tmp/ipykernel_44387/874482196.py", line 40, in solve_ik
    env.render(render_every=render_every)
  File "/home/rilab/ur_ws/src/llm-uncertainty/llm-uncertainty/model/mujoco_parser.py", line 282, in render
    self.viewer.render()
  File "/home/rilab/.local/lib/python3.10/site-packages/mujoco_viewer/mujoco_viewer.py", line 309, in render
    raise Exception(
Exception: GLFW window does not exist but you tried to render.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rilab/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_

In [70]:
env.viewer.close()

In [31]:
q_array = np.vstack([q_init,q_ik_target])

In [32]:
times, q_traj = get_interp_const_vel_traj(q_array, vel=np.radians(30), HZ=env.HZ)
total_time = times[-1]

In [33]:
total_time

4.645504917247373

In [15]:
q_traj.shape

(1941, 6)

In [18]:
from model.robot_client import RobotClient, ROBOT_JOINTS
import rclpy
from control_msgs.action import FollowJointTrajectory
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
import numpy as np
import sys
from builtin_interfaces.msg import Duration

In [19]:
rclpy.init(args=None)
client = RobotClient()

[INFO] [1695403149.423489725] [minimal_client_async]: Successfully connected to action '/scaled_joint_trajectory_controller/follow_joint_trajectory'


### Capture Pose

In [36]:
test_trajectory = [
    (Duration(sec=7, nanosec=0), q_init),
    # (Duration(sec=6, nanosec=500000000), [-0.1 for j in ROBOT_JOINTS]),
]
print(test_trajectory)
trajectory = JointTrajectory(
    joint_names=[client.tf_prefix + joint for joint in ROBOT_JOINTS],
    points=[
        JointTrajectoryPoint(positions=test_pos, time_from_start=test_time)
        for (test_time, test_pos) in test_trajectory
    ],
)
client.send_request(trajectory)

[(builtin_interfaces.msg.Duration(sec=7, nanosec=0), array([-1.57079633, -2.31186313,  2.14413699,  1.7392206 ,  0.78539816,
       -1.57114539]))]


[INFO] [1695403431.600200648] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'
[INFO] [1695403431.602072621] [minimal_client_async]: Result of action: ClientGoalHandle <id=[187 178 207 171  65 200  78  77 188  54  37 109 176  69  32 178], accepted=True, status=0>


ClientGoalHandle <id=[187 178 207 171  65 200  78  77 188  54  37 109 176  69  32 178], accepted=True, status=0>

### IK Pose

In [35]:
sec = int(total_time)
nanosec = int((total_time - sec) * 1e9)
test_trajectory = [
    (Duration(sec=sec, nanosec=nanosec), q_ik_target),
    # (Duration(sec=6, nanosec=500000000), [-0.1 for j in ROBOT_JOINTS]),
]
print(test_trajectory)
trajectory = JointTrajectory(
    joint_names=[client.tf_prefix + joint for joint in ROBOT_JOINTS],
    points=[
        JointTrajectoryPoint(positions=test_pos, time_from_start=test_time)
        for (test_time, test_pos) in test_trajectory
    ],
)
client.send_request(trajectory)

[(builtin_interfaces.msg.Duration(sec=4, nanosec=645504917), array([ 2.31113657e-01, -7.72352027e-01,  1.46417924e+00, -6.93160087e-01,
        1.80070425e+00, -3.39025738e-04]))]


[INFO] [1695403419.694076387] [minimal_client_async]: Sending goal to action server '/scaled_joint_trajectory_controller/follow_joint_trajectory'
[INFO] [1695403419.696910767] [minimal_client_async]: Result of action: ClientGoalHandle <id=[ 91 135  83 170  68 154  73 154 131 170  11  33 229 230   4 243], accepted=True, status=0>


ClientGoalHandle <id=[ 91 135  83 170  68 154  73 154 131 170  11  33 229 230   4 243], accepted=True, status=0>

### Gripper

In [22]:
from pymodbus.client.sync import ModbusTcpClient
from model.gripper import *


In [23]:
graspclient = ModbusTcpClient('192.168.0.13') 
# closeGrasp(200,100,graspclient)
openGrasp(400,1000,graspclient)